- Pre-Requisite: Need to have original "Curve Header Data" populated
- Load "Curve Header Data" tab as ws
- Loop through all curve tabs, and create copy of relevant entry from "Curve Header Data" entry

#### File Setup

In [4]:
import os
import openpyxl
import pandas as pd

In [5]:
fileDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\12 NBS Curve PSD Separation\12.02 Output Files"
file_to_change = r"Populated Curve PSD.xlsx"
filePath = os.path.join(fileDir, file_to_change)

tabName = "Curve Header Data"

In [6]:
# Creates dataframe from excel tab of interest
raw_data = pd.read_excel(filePath, sheet_name=tabName, header=8)

In [7]:
# Create list of all new curve numbers
wb = openpyxl.load_workbook(filePath, data_only=True)

new_curves_list = [ sheet.title for sheet in wb.worksheets if type(sheet.title)==str ] 

#### Main Code
- Loop through each curve number entry in curve header data tab
- For each matching trim worksheet found, append duplicate entry to new dataframe, but rename curve number to match

In [8]:
rows_list = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later

for _, col_data in raw_data.iterrows():
    if type(col_data['Curve number']) == str:
        for curve_number in new_curves_list:
            if col_data['Curve number'] in curve_number:
                mydict = col_data.to_dict()
                mydict.update({'Curve number':curve_number})
                rows_list.append(mydict)

# Concatenating list of dfs to single dfs.
updated_data = pd.DataFrame(rows_list)

#### Need to write updated_data to excel "curve Header Tab"
- May need to append / overlay on existing tab

In [9]:
# Write resulting dataframes to new sheet in PSD with changes.
psd_startrow = 10

with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
    updated_data.to_excel(writer, sheet_name=tabName, index=False, header=False, startrow=psd_startrow)

#### Update Curve Family Cell B7